# Text Generation with Recurrent Neural Networks (RNNs)

In this assignment, you'll build upon your understanding of RNNs and Keras to develop a word-level text generation model.  Your goal is to train a model that learns the stylistic nuances of a chosen corpus and generates new, original text segments that echo the source material's essence.

**Datasets**

We've provided several intriguing text corpora to get you started:

*   Mark Twain
*   Charles Dickens
*   William Shakespeare

**Feel free to explore!**  If you have a particular passion for another author, genre, or a specific text, you're encouraged to use your own dataset of raw text.

In [1]:
# Check if we have a GPU available
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU available. If you're on Colab, go to Runtime > Change runtime and select a GPU hardware accelerator.")

No GPU available. If you're on Colab, go to Runtime > Change runtime and select a GPU hardware accelerator.


In [2]:
def download_file(url, file_path):
    import requests
    r = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(r.content)

def load_dataset(file_path, fraction=1.0):
    with open(file_path, 'r', encoding='utf-8') as f:
        raw_text = f.read()
    return raw_text[:int(fraction * len(raw_text))]

dataset = 'mark_twain.txt' # Other options are mark_twain.txt, charles_dickens.txt

download_file(f'https://github.com/UofT-DSI/deep_learning/raw/main/assignments/downloaded_books/' + dataset, dataset)

# Load chosen dataset. NOTE: If Colab is running out of memory, change the `fraction` parameter to a value between 0 and 1 to load less data.
text = load_dataset(dataset,fraction=1)

# 1. Data Preparation (Complete or Incomplete)

Before we can begin training an RNN model, we need to prepare the dataset. This involves cleaning the text, tokenizing words, and creating sequences the model can be trained on.

## 1.1 Data Exploration

Print the first 1000 characters of the dataset. Report the dataset's size and the number of unique characters it contains.

In [3]:
# Your code here

# Print the first 1000 characters of the dataset
print(text[:1000])

# Report the dataset's size and the number of unique characters
dataset_size = len(text)
unique_chars = len(set(text))
print(f"Dataset size: {dataset_size}")
print(f"Number of unique characters: {unique_chars}")








<!DOCTYPE html>
<html
  lang="en"
  
  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"
  data-a11y-animated-images="system" data-a11y-link-underlines="true"
  >



  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">
  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>
  <link rel="preconnect" href="https://avatars.githubusercontent.com">

  

  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-efd2f2257c96.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-6b1e37da2254.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymous" media

## 1.2 Text Pre-Processing

To prepare the dataset for training, we need to clean the text and create a numerical representation the model can interpret. Perform the following pre-processing steps:

*   Convert the entire text to lowercase.
*   Use the `Tokenizer` class from the `keras.preprocessing.text` module to tokenize the text. You should fit the tokenizer on the text and then convert the text to a sequence of numbers. You can use the `texts_to_sequences` method to do this.

**Note**:
* You'll need to specify an appropriate size for the vocabulary. The number of words in the list of most common words can serve as a guide - does it seem like a reasonable vocabulary size?
* Some of the words will be excluded from the vocabulary, as they don't appear often enough. It's important to provide a value for `oov_token` when creating the Tokenizer instance, so that these words can be represented as "unknown".

In [4]:
from keras.preprocessing.text import Tokenizer
import requests
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# Your code here
VOCAB_SIZE = None
OOV_TOKEN = None

tokenizer = None

# Convert text to lowercase
text = text.lower()

# Initialize and fit the tokenizer
VOCAB_SIZE = 25000
OOV_TOKEN = "<OOV>"
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
tokenizer.fit_on_texts([text])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences([text])[0]

# Print the first 10 words in the vocabulary
print(list(tokenizer.word_index.items())[:10])
print(len(sequences))


[('<OOV>', 1), ('0', 2), ('1', 3), ('quot', 4), ('75', 5), ('25', 6), ('2', 7), ('class', 8), ('3', 9), ('link', 10)]
31605


If everything worked, the following line should show you the first 10 words in the vocabulary:

In [5]:
try:
    print(list(tokenizer.word_index.items())[:10])
except AttributeError:
    print("Tokenizer has not been initialized. Possible issue: Complete the relevant section of the assignment to initialize it."
)


[('<OOV>', 1), ('0', 2), ('1', 3), ('quot', 4), ('75', 5), ('25', 6), ('2', 7), ('class', 8), ('3', 9), ('link', 10)]


## 1.3 Sequence Generation

Now that the text has been tokenized, we need to create sequences the model can be trained on. There are two parts to this:

*   Use the `texts_to_sequences` method from the tokenizer to convert the text to a list of sequences of numbers.
*   Generate the training sequences. Each training sequence should contain `SEQ_LENGTH` token IDs from the text. The target token for each sequence should be the word that follows the sequence in the text.

In [6]:
SEQ_LENGTH = 50  # Choose an appropriate sequence length

# Convert the text to a list of sequences of numbers
sequences = tokenizer.texts_to_sequences([text])[0]

# Generate the training sequences
X = []
y = []
for i in range(SEQ_LENGTH, len(sequences)):
    X.append(sequences[i - SEQ_LENGTH:i])
    y.append(sequences[i])

# Convert lists to numpy arrays for compatibility with Keras
X = np.array(X)
y = np.array(y)

if OOV_TOKEN in tokenizer.word_index:
        mask = y != tokenizer.word_index[OOV_TOKEN]
        X = X[mask]
        y = y[mask]
        
# Print some details to verify the raw target tokens
print(f"First 10 raw target tokens (before one-hot encoding): {y[:10]}")

# Optionally, convert y to one-hot encoding if needed for classification
num_classes = len(tokenizer.word_index) + 1  # Ensure the correct number of classes
y = to_categorical(y, num_classes=num_classes)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_trainval shape: {X_val.shape}')
print(f'y_trainval shape: {y_val.shape}')




First 10 raw target tokens (before one-hot encoding): [  10  122  674  675   33   24   14 1617 1618 1619]
X_train shape: (25244, 50)
y_train shape: (25244, 5837)
X_trainval shape: (6311, 50)
y_trainval shape: (6311, 5837)


Assuming your sequences are stored in `X` and the corresponding targets in `y`, the following line should print the first training sequence and its target:

In [7]:
if len(X) > 0 and len(y) > 0:
    target_index = np.argmax(y[0])
    print(f'Sequence: {X[0]}\nTarget: {target_index}')
    print(f'Translated back to words: {[tokenizer.index_word[i] for i in X[0]]} -> {tokenizer.index_word[target_index]}')
else:
    print("Training sequences have not been generated. Possible issue: Complete the relevant section of the assignment to initialize it.")

Sequence: [1613  463  463 1119  878   11   52 1120  179   11  250  259  250   11
  203  259  203   11 1121 1614 1122 1615   11 1121   10 1616   17 1123
   82  464  465   32   10  122  674  675   33   24   14   67   25   10
  122  674  675   33   24 1124  879   25]
Target: 10
Translated back to words: ['doctype', 'html', 'html', 'lang', 'en', 'data', 'color', 'mode', 'auto', 'data', 'light', 'theme', 'light', 'data', 'dark', 'theme', 'dark', 'data', 'a11y', 'animated', 'images', 'system', 'data', 'a11y', 'link', 'underlines', 'true', 'head', 'meta', 'charset', 'utf', '8', 'link', 'rel', 'dns', 'prefetch', 'href', 'https', 'github', 'githubassets', 'com', 'link', 'rel', 'dns', 'prefetch', 'href', 'https', 'avatars', 'githubusercontent', 'com'] -> link


And the following code will transform y into a one-hot encoded matrix, and split everything into training and validation sets:

In [8]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np

# Ensure that tokenizer has been initialized
if tokenizer is not None:
    # Convert X and y to numpy arrays
    X = np.array(X)
    y = np.array(y)

    # One last thing: let's drop any examples where the target is the OOV token - we don't want our model to predict that (boring!)
    if OOV_TOKEN in tokenizer.word_index:
        mask = y != tokenizer.word_index[OOV_TOKEN]
        X = X[mask]
        y = y[mask]

    # One-hot encode the target token
    y = to_categorical(y, num_classes=VOCAB_SIZE)

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

    print(f'X_train shape: {X_train.shape}')
    print(f'y_train shape: {y_train.shape}')
else:
    print("Tokenizer has not been initialized. Please initialize it and load the vocabulary before continuing.")


IndexError: boolean index did not match indexed array along dimension 1; dimension is 50 but corresponding boolean dimension is 5837

# 2. Model Development (Complete or Incomplete)

With the dataset prepared, it's time to develop the RNN model. You'll need to define the architecture of the model, compile it, and prepare it for training.

## 2.1 Model Architecture

Define the architecture of your RNN model. You can design it however you like, but there are a few features that it's important to include:

*   An embedding layer that learns a dense representation of the input tokens. You'll need to specify the input dimension (the size of the vocabulary) and the output dimension (the size of the dense representation). Remember, you can look at the documentation [here](https://keras.io/api/layers/core_layers/embedding/).
*   At least one recurrent layer. We have learned how to use LSTM layers in class, but you can use other types of recurrent layers if you prefer. You can find the documentation [here](https://keras.io/api/layers/recurrent_layers/lstm/).
*   A dense layer with a softmax activation function. This layer will output a probability distribution over the vocabulary, so that the model can make predictions about the next token.

In [12]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Flatten

model = Sequential([
    Embedding(VOCAB_SIZE, 128, input_length=SEQ_LENGTH),
    LSTM(256, return_sequences=True),
    Dropout(0.3),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

# Train the model


# Check if the model has layers before trying to print the summary
if len(model.layers) > 0:
    model.summary()
else:
    print("No layers have been added to the model. Please complete the assignment by adding the required layers.")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 128)           3200000   
                                                                 
 lstm_1 (LSTM)               (None, 50, 256)           394240    
                                                                 
 dropout_2 (Dropout)         (None, 50, 256)           0         
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               3277056   
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 5837)             

## 2.2 Model Compilation

Compile the model with an appropriate loss function and optimizer. You might also want to track additional metrics, such as accuracy.

Give a short explanation of your choice of loss function and optimizer:

_your explanation here_

In [13]:
# Your code here
# Adam combines the benefits of two other popular optimization algorithms - AdaGrad and RMSProp, requires less tuning and ismore efficient. 
# Caterogical crossentropy because texts are one hot encoded which makes it suitable for tasks where the model needs to output a probability distribution over multiple classes


# Compile the model
from keras.optimizers import Adam 
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


## 2.3 Model Training

Train the model on the training data you've prepared.

* Train your model for 5 epochs with a batch size of 128. Use the validation data for validation.
* Store the training history in a variable called `history`.

In [14]:
# Your code here
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

#early stopping to monitor the validation loss and avoid overfitting
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

#reducing learning rate on plateau
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience= 5, factor= 0.2, min_lr= 1e-5)

history = model.fit(X_train, y_train, epochs=15, batch_size=128, validation_data=(X_val, y_val),callbacks=[early_stop,rlrop])



Epoch 1/15
198/198 [==============================] - 109s 545ms/step - loss: 6.7947 - accuracy: 0.1007 - val_loss: 5.9886 - val_accuracy: 0.1247 - lr: 1.0000e-04
Epoch 2/15
198/198 [==============================] - 136s 684ms/step - loss: 5.8267 - accuracy: 0.1184 - val_loss: 5.7645 - val_accuracy: 0.1263 - lr: 1.0000e-04
Epoch 3/15
198/198 [==============================] - 109s 552ms/step - loss: 5.5721 - accuracy: 0.1229 - val_loss: 5.7003 - val_accuracy: 0.1307 - lr: 1.0000e-04
Epoch 4/15
198/198 [==============================] - 109s 554ms/step - loss: 5.3832 - accuracy: 0.1280 - val_loss: 5.6817 - val_accuracy: 0.1325 - lr: 1.0000e-04
Epoch 5/15
198/198 [==============================] - 152s 770ms/step - loss: 5.2525 - accuracy: 0.1301 - val_loss: 5.6725 - val_accuracy: 0.1355 - lr: 1.0000e-04
Epoch 6/15
198/198 [==============================] - 148s 750ms/step - loss: 5.1432 - accuracy: 0.1333 - val_loss: 5.6648 - val_accuracy: 0.1412 - lr: 1.0000e-04
Epoch 7/15
198/198 [==

Plot the training history to visualize the model's learning progress. Your plot should include the training and validation loss.

In [15]:
import matplotlib.pyplot as plt

# Your code here

# Plot the training history to visualize the model's learning progress
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

# 3. Text Generation (Complete or Incomplete)

Write a method called `generate_text` that uses the trained model to generate new text. The method should take the following parameters:

*   `model`: The trained RNN model.
*   `tokenizer`: The tokenizer used to pre-process the text data.
*   `seed_text`: The seed text the model will use to generate new text.
*   `max_sequence_len`: The maximum length of the sequence used to generate new text.

The method should return the generated text.

An overview of the text generation process you should follow:

1. Tokenize the seed text using the tokenizer we built before.
2. Pad the sequences to the same length as the training sequences - you can use the `pad_sequences` method from the `keras.preprocessing.sequence` module, which is documented [here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences).
3. Use the model to predict the next token in the sequence. Remember that the model will output a probability distribution over the vocabulary, so you'll need to use `np.argmax` to find the token with the highest probability.
4. Add the predicted token to the sequence and remove the first token.
5. Repeat steps 3-4 until you have generated the desired number of tokens.
6. Convert the generated token IDs back to words and return the combined result as a single string.

This is a challenging task, so don't hesitate to ask for help if you need it. It's okay if the generated text doesn't make much sense yet - we'll work on improving the model next.
As a bonus, you can make your method generate "gpt-style" by having it print out each word as it's generated, so you can see the text being generated in real time.

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def get_predicted_word(model, sequence):
    """
    Get the predicted word from the model.
    This helper function uses a concept we haven't covered in class yet: temperature.
    In essence, a little bit of randomness in what word we predict can make the text more interesting.
    """

    # Use the model to predict the next token in the sequence
    yhat = model.predict(sequence, verbose=0)

    # Get the index of the predicted word, according to the probabilities
    yhat = np.argmax(yhat, axis=-1)

    return yhat
def sample_with_temperature(preds, temperature=1.0):
    """
    Helper function to sample an index from a probability array reweighted by temperature.
    """
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(model, tokenizer, seed_text, max_sequence_len, temperature=1.0):

    """
    Generate new text using the trained model.
    You can use the `get_predicted_word` helper function to help you with this.
    """    
    input_text = seed_text.lower()
    result=[]

    for _ in range(max_sequence_len):
        token_list = tokenizer.texts_to_sequences([input_text])[0]
        token_list = pad_sequences([token_list], maxlen=SEQ_LENGTH, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted = sample_with_temperature(predicted_probs, temperature)
        output_word = tokenizer.index_word.get(predicted, OOV_TOKEN)
        input_text += ' ' + output_word
        result.append(output_word)
    return ' '.join(result)


In [24]:
# Test the text generation function
generate_text(model, tokenizer, 'hamlet', SEQ_LENGTH)

print(generate_text(model, tokenizer, 'hamlet', SEQ_LENGTH))

7 community cb61ixzf9xm 7i4waaabahrstlmaererererereiiiizmzmzm0rererererererevvvvvvvvzmzmzmzmznd3d3d3d3eiiiiiiiiimzmzmzmzqqqqqqqqqqqqu7u7u7u7u7u7u7u7u8zmzmzmzmzm3d3d3d3d3d3d3d3d3d3d3d3d3d3d3d3d3d3d7u7u7u7u7u7u7u7u7u7u src el8zsq3cxncdn8ze9anenotv bg jtml community 3f iuckiopdoc5mhin k8jwyyw4dbvfry5lffbqrvxj2 f z event wcrujuiepmh q 2fuoft g6agu3icegbawlbd2ciovtxqyzbojo3mgwiiynhkas5id searchsuggestionsdialog 5il73po0vpbzkwgrh2vhmfdco5na1ypljhwgmxxy sap0cvontth4rxl9onvrfjvumysdepf ael 3mw3xwjrql0ol exk0ex9pjk3r6ih4 cookie expected 0 auto da f0v9unufx 18dp ksux8hsatvc9wipyzagg fxbmtkggwuafyoonk59du955s nmkb5szjjzjnedguh6lpuk6btfivdiqwfwqr0avszz2g1m6fxdlryoquzzeqsvessau 7 tqvwjyt57 xkqogy7nwqqoq09hqsi9lzbngqdptln1oooq0vrpciremsdz a nuly9d2klkkmjk8t96yiyw8p3lk5ev j20 ieulhg483f9ug1alejzswkn3dz2xbativ1cxbden6b4ce1tc9gbtp 3vzevlfuq39ltvikggezvfzwhu999tmtdzitza6pt8toznlzuwbawxk51hogkwo1lypkkooqyst a5cxumue9ujhuhwirn2yblslplamf6v25h5s rsqna0gotg1i65adxuegpbxyhgkf14j2t6fbnrp29si1 kgan2zifbdxr2bjr

It's likely that the text generated by your model doesn't make much sense yet. This is because the model hasn't been trained for very long, and the training dataset is relatively small. 

# 4. Model Refinement (Complete or Incomplete)

In this last section, you'll work on improving your model. There are many ways to do this, but here are a few ideas to get you started:

* Use pre-trained embeddings: the code below will help you to load pre-trained embeddings through Keras. 
* Experiment with different model architectures, including the number of layers, the number of units in each layer, and the use of dropout layers.
* Train your model for longer. You can also experiment with different batch sizes.

Implement and test out at least one of these ideas. If you have other ideas for improving the model, feel free to try them out as well.

In [28]:
import requests
import zipfile
import os

!wget https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
!unzip glove.6B.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [29]:
# Load the pre-trained embeddings
embeddings_index = {}
try:
    with open('glove.6B.100d.txt', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print(f'Found {len(embeddings_index)} word vectors.')
except FileNotFoundError:
    print("'glove.6B.100d.txt' File not found. Please make sure you have ran the previous cell.")

Found 400000 word vectors.


In [30]:
# Check if VOCAB_SIZE is set
if VOCAB_SIZE is None:
    print("You need to complete the previous parts of your assignment in order for this to work.")
else:
    # Create an embedding matrix
    embedding_matrix = np.zeros((VOCAB_SIZE, 100))
    for word, i in tokenizer.word_index.items():
        if i < VOCAB_SIZE:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector

In [31]:
# Check if VOCAB_SIZE is set
if VOCAB_SIZE is None:
    print("You need to complete the previous parts of your assignment in order for this to work.")
else:
    embedding_layer = Embedding(
        VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=SEQ_LENGTH, trainable=False
    )

In [37]:
# Define a new model and train it

# Your code here

from keras.layers import Embedding, LSTM, Dense, Dropout, GRU, Bidirectional, LayerNormalization

new_model = Sequential([
        # Embedding layer with pre-trained GloVe embeddings
        embedding_layer,

        # Bidirectional LSTM layer
        Bidirectional(LSTM(256, return_sequences=True)),
        Dropout(0.3),
        LayerNormalization(),
        
        # Additional Bidirectional LSTM layer
        Bidirectional(LSTM(256, return_sequences=True)),
        Dropout(0.3),
        LayerNormalization(),
        
        # GRU layer
        GRU(256),
        Dropout(0.3),
        
        # Dense layers
        Dense(256, activation='relu'),
        Dropout(0.3),
        LayerNormalization(),
        
        Dense(VOCAB_SIZE, activation='softmax')
    ])
learning_rate = 0.0001  # Adjust this value as needed
optimizer = Adam(learning_rate=learning_rate)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
new_history = model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_val, y_val), callbacks=[rlrop])




Epoch 1/50
198/198 [==============================] - 112s 559ms/step - loss: 4.4361 - accuracy: 0.1838 - val_loss: 5.4200 - val_accuracy: 0.1932 - lr: 1.0000e-04
Epoch 2/50
198/198 [==============================] - 151s 761ms/step - loss: 4.3090 - accuracy: 0.1941 - val_loss: 5.4658 - val_accuracy: 0.1916 - lr: 1.0000e-04
Epoch 3/50
198/198 [==============================] - 154s 777ms/step - loss: 4.1893 - accuracy: 0.1971 - val_loss: 5.5788 - val_accuracy: 0.1992 - lr: 1.0000e-04
Epoch 4/50
198/198 [==============================] - 144s 728ms/step - loss: 4.0614 - accuracy: 0.2003 - val_loss: 5.7039 - val_accuracy: 0.2055 - lr: 1.0000e-04
Epoch 5/50
198/198 [==============================] - 135s 680ms/step - loss: 3.9352 - accuracy: 0.2092 - val_loss: 5.8589 - val_accuracy: 0.2122 - lr: 1.0000e-04
Epoch 6/50
198/198 [==============================] - 147s 743ms/step - loss: 3.8083 - accuracy: 0.2183 - val_loss: 6.0871 - val_accuracy: 0.2190 - lr: 1.0000e-04
Epoch 7/50
198/198 [==

## Criteria

|Criteria|Complete|Incomplete|
|----|----|----|
|Task 1|The task has been completed successfully and there are no errors.|The task is still incomplete and there is at least one error.|
|Task 2|The task has been completed successfully and there are no errors.|The task is still incomplete and there is at least one error.|
|Task 3|The task has been completed successfully and there are no errors.|The task is still incomplete and there is at least one error.|
|Task 4|The task has been completed successfully and there are no errors.|The task is still incomplete and there is at least one error.|

## Submission Information

🚨**Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)**🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/deep_learning/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.